_from Dani Cluff in #stats_

`
Hi there - Do we know how many events were hosted on the Qgiv platform (including standard events, auctions, and P2P) in 2022 vs. 2023? If not, can we pull these numbers?
`

- Pull active, distinct auction form IDs from transactions, grouped by year
- Pull active, distinct P2P form IDs from transactions, grouped by year
- Pull average events from analyticsqgiv_monthly for 2022+, group by year, aggregate on max, mean, & median
- Merge all by year, calculate YoY

In [2]:
import sys
sys.path.insert(1, '../../scripts/')
from s3_support import *

import pandas as pd

In [22]:
q = '''select 
            year, 
            count(distinct(form)) as p2p
        from transactions 
        where
            status='A' and
            source='p2p' and
            year>=2022
        group by year'''
p2p_count = redshift_query_read(q, schema='production')

# form.type=5 for auctions
q = '''select 
            t.year, 
            count(distinct(t.form)) as auctions
        from transactions as t
        left join form as f on t.form=f.id
        where
            t.status='A' and
            f.type=5 and
            t.year>=2022
        group by year'''
auction_count = redshift_query_read(q, schema='production')

In [12]:
q = '''select
            date as month,
            sum(events_count) as qgiv_events
        from analyticsqgiv_monthly
        where date>='2022-01-01'
        group by date'''
qgiv_event_count = redshift_query_read(q, schema='public')

In [24]:
print("Qgiv events:")
qgiv_event_count['year'] = pd.to_datetime(qgiv_event_count['month']).dt.year
standard_events_count = qgiv_event_count.groupby('year')['qgiv_events'].agg(['max', 'mean', 'median']).reset_index()
standard_events_count

Qgiv events:


,year,max,mean,median
0,2022,12474,11817.250000,11850.5
1,2023,16429,15251.545455,15120.0
2,2024,19142,17806.285714,17969.0


In [25]:
cols = ['year', 'standard events max',
        'standard events mean', 'standard events median']
standard_events_count.columns = cols

In [16]:
print("P2P events:")
p2p_count

P2P events:


,year,p2p
0,2023,2450
1,2022,2258
2,2024,1983


In [30]:
df = auction_count.merge(p2p_count, on='year')
df = df.merge(standard_events_count, on='year')
df.sort_values('year', ascending=True, inplace=True)

for col in df.columns:
    if col != 'year':
        df['{} YoY'.format(col)] = df[col].pct_change()
        
df

,year,auctions,p2p,standard events max,standard events mean,standard events median,auctions YoY,p2p YoY,standard events max YoY,standard events mean YoY,standard events median YoY
2,2022,657,2258,12474,11817.250000,11850.5,NaN,NaN,NaN,NaN,NaN
1,2023,812,2450,16429,15251.545455,15120.0,0.235921,0.085031,0.317059,0.290617,0.275896
0,2024,788,1983,19142,17806.285714,17969.0,-0.029557,-0.190612,0.165135,0.167507,0.188426
